In [1]:
import pandas as pd
import numpy as np
import random
import itertools

col_list = ["SERIALNO","SEX", "RAC1P", "AGEP","SCHL","HINCP","VEH","NP","WIF"]
df = pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\pums\\temp_pums.csv", usecols=col_list)

df.head(5)
df=df.fillna(df.median())

C:\Users\abubabu\AppData\Local\Temp\ipykernel_34820\187016552.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df=df.fillna(df.median())


#### marginal functions

In [2]:
def get_target_marginals(d):
    factors = list(d.keys())
    targets = [sorted([(k2, v2) for k2, v2 in v.items()]) for k, v in d.items()]
    targets = np.array([[v for _, v in item] for item in targets])
    return factors, targets

def get_table(df, targets):
    factors, target_marginals = get_target_marginals(targets)

    cross_tab = pd.crosstab(df[factors[0]], [df[c] for c in factors[1:]], dropna=False)
    shape = tuple([df[c].unique().shape[0] for c in factors])
    print(shape)

    table = cross_tab.values.reshape(shape)
    
    return factors, target_marginals, table


#### table = contigency table

Loading data from census

In [3]:
#ddg = demographic_data_groups
ddg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_demographic.csv")
edg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_economic.csv")
hdg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_housing.csv")
sdg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_social.csv")
edudg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_educational.csv")


#### fetching margin values from demographic data groups 

In [4]:
temp_race = {
    'asian': 'DP05_0067E',
    'black': 'DP05_0065E',
    'native': 'DP05_0066E',
    'other': 'DP05_0069E',
    'pacific-islander': 'DP05_0068E',
    'white': 'DP05_0064E'
}
temp_sex = {'female': 'DP05_0003E', 'male': 'DP05_0002E'}
temp_age = {
    'adult': ddg.loc[ddg['id'] == 'DP05_0010E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0011E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0012E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0013E', 'value'].values[0],
            
    'child': ddg.loc[ddg['id'] == 'DP05_0005E', 'value'].values[0],

    'senior': ddg.loc[ddg['id'] == 'DP05_0014E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0015E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0016E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0017E', 'value'].values[0],

    'tenager': ddg.loc[ddg['id'] == 'DP05_0006E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0007E', 'value'].values[0] +
            ddg.loc[ddg['id'] == 'DP05_0008E', 'value'].values[0],

    'young-adult': ddg.loc[ddg['id'] == 'DP05_0009E', 'value'].values[0]
}
temp_inc = {
    '0': int(edg.loc[edg['id'] == 'DP03_0076E', 'value'].values[0]),
    '1': int(edg.loc[edg['id'] == 'DP03_0076E', 'value'].values[0]),
    '2': int(edg.loc[edg['id'] == 'DP03_0077E', 'value'].values[0]),
    '3': int(edg.loc[edg['id'] == 'DP03_0078E', 'value'].values[0]),
    '4': int(edg.loc[edg['id'] == 'DP03_0079E', 'value'].values[0]),
    '5': int(edg.loc[edg['id'] == 'DP03_0080E', 'value'].values[0]),
    '6': int(edg.loc[edg['id'] == 'DP03_0081E', 'value'].values[0]),
    '7': int(edg.loc[edg['id'] == 'DP03_0082E', 'value'].values[0]),
    '8': int(edg.loc[edg['id'] == 'DP03_0083E', 'value'].values[0]),
    '9': int(edg.loc[edg['id'] == 'DP03_0084E', 'value'].values[0]),
    '10': int(edg.loc[edg['id'] == 'DP03_0085E', 'value'].values[0])

}
temp_veh={
    '1' : int(hdg.loc[hdg['id'] == 'DP04_0059E', 'value'].values[0]),
    '2' : int(hdg.loc[hdg['id'] == 'DP04_0060E', 'value'].values[0]),
    '3' : int(hdg.loc[hdg['id'] == 'DP04_0061E', 'value'].values[0]),
    '0' : int(hdg.loc[hdg['id'] == 'DP04_0058E', 'value'].values[0]),
}

temp_np={ #counted from pums
    '2': 13904,
    '1': 7603,
    '4': 7276,
    '3': 5979,
    '5': 5170,
    '6': 2328,
    '7': 1036,
    '8': 424,
    '9': 288,
    '11': 143,
    '10': 120,
    '15': 45,
    '20': 40,
    '17': 34,
    '16': 32,
    '12': 24,
    '14': 14
}
temp_educ={
    'Bachelor': int(edudg.loc[edudg['id'] == 'S1501_C01_005E', 'value'].values[0]) +
                int(edudg.loc[edudg['id'] == 'S1501_C01_012E', 'value'].values[0]),
    'Graduate': int(edudg.loc[edudg['id'] == 'S1501_C01_013E', 'value'].values[0]),
    'High-school': int(edudg.loc[edudg['id'] == 'S1501_C01_003E', 'value'].values[0]) +
                int(edudg.loc[edudg['id'] == 'S1501_C01_009E', 'value'].values[0]) +
                int(edudg.loc[edudg['id'] == 'S1501_C01_010E', 'value'].values[0]),
    'below-high': int(edudg.loc[edudg['id'] == 'S1501_C01_002E', 'value'].values[0]) +
                int(edudg.loc[edudg['id'] == 'S1501_C01_007E', 'value'].values[0]) +
                int(edudg.loc[edudg['id'] == 'S1501_C01_008E', 'value'].values[0]),
    'college': int(edudg.loc[edudg['id'] == 'S1501_C01_004E', 'value'].values[0]) +
            int(edudg.loc[edudg['id'] == 'S1501_C01_011E', 'value'].values[0])
}
temp_wif={
    '0': 31913652,
    '1': 45272984,
    '2': 35065006,
    '3': 8504406
}
margin_dict = {
    'HINCP':temp_inc,
    'VEH': temp_veh,
    'NP' : temp_np,
    'WIF': temp_wif,
}


In [5]:
f, u, X = get_table(df, margin_dict)

(11, 4, 17, 4)


C:\Users\abubabu\AppData\Local\Temp\ipykernel_34820\170147130.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  targets = np.array([[v for _, v in item] for item in targets])


X = contigency table\
u = target marginals\
f = helper vector

In [6]:
X=np.nan_to_num(X, nan=0.0).astype('float64')

In [7]:
def get_coordinates(M):
    return list(itertools.product(*[list(range(n)) for n in M.shape]))

# returns a dict with marginals for each unique value of dimention i in M
def get_marginals(M, i):
    coordinates = get_coordinates(M)
    key = lambda tup: tup[0]
    counts = [(c[i], M[c]) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: sum([v[1] for v in g]) for k, g in counts}

    return counts

def get_all_marginals(M):
    return np.array([[v for _, v in get_marginals(M, i).items()]
                     for i in range(len(M.shape))])

# returns a dict with counts for each unique value of dimention i in M
def get_counts(M, i):
    coordinates = get_coordinates(M)

    key = lambda tup: tup[0]
    counts = [(c[i], M[c], c) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: [(tup[1], tup[2]) for tup in g] for k, g in counts}

    return counts

#ipu to update values 
def update_values(M, i, u):
    marg = get_marginals(M, i)
    vals = get_counts(M, i)
    d = [[(c, n * u[k] / marg[k]) for n, c in v] for k, v in vals.items()]
    d = itertools.chain(*d)
    d = list(d)

    return d

def ipf_update(M, u):
    for i in range(len(M.shape)):
        values = update_values(M, i, u[i])
        for idx, v in values:
            # print(idx)
            M[idx] = v

    o = get_all_marginals(M)
    d = get_deltas(o, u)

    return M, d

# o-t
def get_deltas(o, t):
    return np.array([np.linalg.norm(np.array(o[r]) - np.array(t[r]), 2) for r in range(o.shape[0])])


def get_weights(X, max_iters=50, zero_threshold=0.0001, convergence_threshold=3, debug=False):
    M = X.copy()

    d_prev = np.zeros(len(M.shape))
    count_zero = 0

    for _ in range(max_iters):
        # print(_,count_zero)
        # print(u)
        M, d_next = ipf_update(M, u)
        d = np.linalg.norm(d_prev - d_next, 2)

        if d < zero_threshold:
            count_zero += 1

        if debug:
            print(','.join([f'{v:.5f}' for v in d_next]), d)
        d_prev = d_next

        # breaks if zero threshold triggered in 3 consecutive run
        if count_zero >= convergence_threshold:
            break

    w = M / M.sum()
    return M,w

In [8]:
M,w = get_weights(X)

C:\Users\abubabu\AppData\Local\Temp\ipykernel_34820\4157681652.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[v for _, v in get_marginals(M, i).items()]


In [10]:
# file=open("temp.txt",'w')
# file.write(str(M))
M.max()
constf=open('hhold_constrains.txt','w')
constf.write(' '.join(str(v) for v in M.flatten()))
constf.close()

<b> Sampling </b>

In [24]:
import functools

def get_sampling_weights(df, f, w):
    get_filters = lambda df, fields, values: [df[f] == v for f, v in zip(fields, values)]
    get_total = lambda df, fields, values: df[functools.reduce(lambda a, b: a & b, get_filters(df, fields, values))].shape[0]

    return {k: v / get_total(df, f, k) for k, v in zip(list(itertools.product(*[sorted(df[c].unique()) for c in f])), np.ravel(w))}

def get_samples(df, f, w, n=10000):
    file=open("temp.txt",'w')
    weights = get_sampling_weights(df, f, w)
    file.write(str(weights))
    file.close()
    s = df.apply(lambda r: weights[tuple([r[c] for c in f])], axis=1)
    return df.sample(n=n, replace=True, weights=s)

sample_df = get_samples(df, f, w)
sample_df

C:\Users\abubabu\AppData\Local\Temp\ipykernel_30296\3899826299.py:7: RuntimeWarning: invalid value encountered in double_scalars
  return {k: v / get_total(df, f, k) for k, v in zip(list(itertools.product(*[sorted(df[c].unique()) for c in f])), np.ravel(w))}


,SERIALNO,AGEP,SCHL,SEX,RAC1P,HINCP,VEH,NP,WIF
944,2017000150521,young-adult,Graduate,female,asian,5,1.0,1.0,2.0
11198,2018HU0278218,adult,High-school,male,white,2,3.0,12.0,1.0
20557,2019HU0390640,adult,Graduate,male,asian,3,1.0,1.0,2.0
11198,2018HU0278218,adult,High-school,male,white,2,3.0,12.0,1.0
11197,2018HU0278218,adult,college,female,white,2,3.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...
2025,2017000342692,adult,college,male,white,7,2.0,9.0,2.0
42494,2021HU1041986,adult,Graduate,female,other,9,1.0,1.0,2.0
33966,2020HU0852554,senior,Bachelor,male,white,10,1.0,9.0,3.0
28415,2020HU0110778,adult,college,male,white,8,2.0,9.0,3.0


In [25]:
sample_df.to_csv("sampled_DATA.csv")

In [28]:
temp=sample_df.groupby(['SERIALNO'],sort=True)

In [29]:
temp.head(30)

,SERIALNO,AGEP,SCHL,SEX,RAC1P,HINCP,VEH,NP,WIF
944,2017000150521,young-adult,Graduate,female,asian,5,1.0,1.0,2.0
11198,2018HU0278218,adult,High-school,male,white,2,3.0,12.0,1.0
20557,2019HU0390640,adult,Graduate,male,asian,3,1.0,1.0,2.0
11198,2018HU0278218,adult,High-school,male,white,2,3.0,12.0,1.0
11197,2018HU0278218,adult,college,female,white,2,3.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...
18352,2019GQ0147094,young-adult,High-school,male,asian,0,2.0,1.0,2.0
43176,2021HU1149480,adult,Bachelor,female,white,9,1.0,6.0,1.0
38783,2021HU0404204,senior,Bachelor,female,asian,9,0.0,2.0,2.0
44667,2021HU1395044,adult,Bachelor,female,other,9,2.0,7.0,1.0


In [37]:
sample_df['SERIALNO'].value_counts()

2018HU0278218    1583
2020HU0966702    1158
2017000342692    1083
2018HU0815113     850
2020HU0852554     615
                 ... 
2019HU1141733       1
2020HU1068045       1
2018HU1191249       1
2018HU0136208       1
2020HU0716652       1
Name: SERIALNO, Length: 108, dtype: int64

Cross-tabulation of resulting sampled matrix

In [ ]:
sample_df.to_csv("SAMPLE_DATA.csv",header=True)
#education

marginals remain kind of the same